# Customizing Models and training algorithms

## Loss Function
En algunos casos quizás por las características de los datos, sea necesario utilizar funciones de costo alternativas, como en este ejemplo donde se simula la implementación de Huber loss function a pesar de que keras contiene tf.keras.losses.Huber. 

En un caso hipotético donde se entrena un modelo de regresión, y luego de realizar la limpieza de outliers de los datos, aún mantienen cierto ruido, por lo cual el error cuadratico medio puede penalizar demasiado a errores largos y generar un modelo impreciso. El error absoluto medio quizás no penaliza tanto a los outliers, pero el entrenamiento pueden demorar en converger, y el modelo entrenado puede ser impreciso también. Por lo tanto se procede a usar una función de pérdida custom en lugar de MSE.


```Python
import tensorflow as tf
import keras

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    max_error = 1
    is_small_error = tf.abs(error) < max_error
    squared_loss = tf.square(error)
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)
```

Para mejor performance se debe utilizar únicamente una implementación vectorial, para beneficiarse de la optimización de TensorFlow solo se deben usar operaciones de TensorFlow

In [5]:
import tensorflow as tf
import keras

tf.random.set_seed(42)

@tf.function
def huber_fn(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    abs_error = tf.abs(error)
    quadratic = 0.5 * tf.square(error)
    linear = delta * (abs_error - 0.5 * delta)
    return tf.where(abs_error <= delta, quadratic, linear)

- Vectorización total: todo opera sobre tensores, sin bucles.
- Multiplica por delta en la parte lineal (más general y matemáticamente correcto).
- Decorador @tf.function: compila la función en una graph function, mejorando performance al ser usado en entrenamiento o producción.

Se utilizará el dataset de Housing para simular un problema de regresión

In [6]:
from tensorflow.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train.shape

(404, 13)

404 muestras con 13 atributos

In [7]:
model = keras.Sequential(
    [
        keras.layers.Dense(32, activation="relu", input_shape=(x_train.shape[1],)),
        keras.layers.Dropout(0.4),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1)
    ])

optimizer = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(
    loss=huber_fn, 
    optimizer=optimizer,
    metrics=["mae", "mse"]  # Métricas apropiadas para regresión, mean absolute error y mean squared error
    #metrics=["accuracy"] Aqui se uso accuracy, pero es para clasificación, en regresión se usa mae o mse
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("custom_model_loss_function.h5", save_best_only=True)

model.fit(
    x_train, 
    y_train, 
    epochs=200, 
    validation_data=(x_test, y_test),
    callbacks=[checkpoint_cb, early_stopping]
)

Epoch 1/200
13/13 [==============================] - 0s 8ms/step - loss: 22.4025 - mae: 22.9025 - mse: 610.4813 - val_loss: 22.8437 - val_mae: 23.3437 - val_mse: 629.6715
Epoch 2/200
13/13 [==============================] - 0s 3ms/step - loss: 22.2264 - mae: 22.7264 - mse: 602.6571 - val_loss: 22.7012 - val_mae: 23.2012 - val_mse: 622.4734
Epoch 3/200
13/13 [==============================] - 0s 3ms/step - loss: 22.0687 - mae: 22.5687 - mse: 595.4377 - val_loss: 22.5678 - val_mae: 23.0678 - val_mse: 615.8667
Epoch 4/200
13/13 [==============================] - 0s 3ms/step - loss: 21.9249 - mae: 22.4249 - mse: 588.6517 - val_loss: 22.4375 - val_mae: 22.9375 - val_mse: 609.5627
Epoch 5/200
13/13 [==============================] - 0s 2ms/step - loss: 21.7781 - mae: 22.2781 - mse: 582.2001 - val_loss: 22.3055 - val_mae: 22.8055 - val_mse: 603.3384
Epoch 6/200
13/13 [==============================] - 0s 3ms/step - loss: 21.6386 - mae: 22.1386 - mse: 575.9510 - val_loss: 22.1678 - val_mae: 22

/opt/anaconda3/envs/homl3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 0s 3ms/step - loss: 21.5349 - mae: 22.0349 - mse: 570.9202 - val_loss: 22.0222 - val_mae: 22.5222 - val_mse: 590.4993
Epoch 8/200
13/13 [==============================] - 0s 3ms/step - loss: 21.3652 - mae: 21.8652 - mse: 563.2833 - val_loss: 21.8597 - val_mae: 22.3597 - val_mse: 583.3077
Epoch 9/200
13/13 [==============================] - 0s 2ms/step - loss: 21.1375 - mae: 21.6375 - mse: 553.4991 - val_loss: 21.6743 - val_mae: 22.1743 - val_mse: 575.2074
Epoch 10/200
13/13 [==============================] - 0s 2ms/step - loss: 20.9574 - mae: 21.4574 - mse: 546.6489 - val_loss: 21.4606 - val_mae: 21.9606 - val_mse: 566.0571
Epoch 11/200
13/13 [==============================] - 0s 2ms/step - loss: 20.7393 - mae: 21.2393 - mse: 538.5350 - val_loss: 21.2081 - val_mae: 21.7081 - val_mse: 555.3569
Epoch 12/200
13/13 [==============================] - 0s 3ms/step - loss: 20.5376 - mae: 21.0376 - mse: 530.4330 - val_loss: 20.9207 - val_mae: 21.4207 - v

### Saving and loading models containing custom components

Con una función costo hay que proveer un diccionario que mapea el nombre de la función a la función en si cuando cargás la misma.

In [9]:
model = tf.keras.models.load_model("custom_model_loss_function.h5",
                            custom_objects={"huber_fn": huber_fn})

Con esta implementación, cualquier error entre -1 y 1 es considerado chico, Pero como configurar un treshold distinto? Una solución es modificar la función custom

In [14]:
def create_huber(treshold=1.0): #Implementacion distinta de la primera, a modo de ejemplo
    def huber_fn(y_true, y_predicted):
        error = y_true - y_pred
        abs_error = tf.abs(error)
        quadratic = 0.5 * tf.square(error)
        linear = treshold * (abs_error - 0.5 * delta)
        return tf.where(abs_error <= treshold, quadratic, linear)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")
model.save("custom_model_loss_function_2.h5")

model = tf.keras.models.load_model("custom_model_loss_function_2.h5",
                                  custom_objects={"huber_fn": create_huber(2.0)})

Cuando se guarda el modelo, el umbral no se va a guardar, esto significa que hay que especificarlo cuando se carga el modelo. También es notable que el nombre de funcion que se le da a keras es huber_fn, no create_huber

Esto se puede resolver creando una subclase de una clase tf.keras.losses.Loss e implementando su método get_config()

In [20]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        abs_error = tf.abs(error)
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * abs_error - self.threshold**2/2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}


model.compile(loss=HuberLoss(2.), optimizer="nadam")
model.save("custom_class_loss_function.h5")

model = tf.keras.models.load_model("custom_class_loss_function.h5",
                                   custom_objects={"HuberLoss": HuberLoss})

El constructor acepta **kwargs y lo pasa al padre cuando hace super(**kwargs), esto es para gestionar los hyperparámetros standard, como el nombre de la perdida y el algoritmo de reducción para usar instancias de perdidad individual. Por defecto esto es auto, que es equivalente a "SUM_OVER_BATCH_SIZE". La perdida sera la suma de las perdidas de la instancia, ponderada por los pesos de las muestras, si este valor existe. Luego se lo divide por el tamaño del batch (no por la suma de pesos, no es un promedio ponderado). Otro posible valor es "SUM" o "NONE"

El método call(), toma las etiquetas y las predicciones, computa las pérdidas de la instancia y las retorna.

El método get_config() retorna un diccionario con cada hyperparámetro y su valor, primero llama al get_config() de la clase padre

Ahora cuando se guarda el modelo, keras llama al método get_config() de la instancia de la clase y guarda la configuración en el SavedModel. Cuando se carga el modelo, llama al método from_config(), que de la clase HuberLoss, crea una instancia de la clase y le pasa **config al constructor. Y con esto se implementan las funciones de pérdida.